In [1]:
!pip install timm flash_attn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 34.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.7 MB/s eta 0:00:00
  Created wheel for flash_attn: filename=flash_attn-2.6.3-cp310-cp310-linux_x86_64.whl size=187315346 sha256=6ebfbdcbdd164f80278a954d29a1bc9d620130264215f0fb93374a6ab4e0a283
  Stored in directory: /root/.cache/pip/wheels/7e/e3/c3/89c7a2f3c4adc07cd1c675f8bb7b9ad4d18f64a72bccdfe826
Successfully built flash_attn


In [2]:
from transformers import AutoProcessor, AutoModelForCausalLM
from PIL import Image
import requests
import copy
%matplotlib inline

In [3]:
model_id = 'microsoft/Florence-2-large'
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True).eval().cuda()
processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)

config.json:   0%|          | 0.00/2.44k [00:00<?, ?B/s]

configuration_florence2.py:   0%|          | 0.00/15.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-large:
- configuration_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_florence2.py:   0%|          | 0.00/127k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-large:
- modeling_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-large:
- processing_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer_config.json:   0%|          | 0.00/34.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.10M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
!pip install torch

import torch

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [5]:
def run_example(task_prompt, text_input=None):
    if text_input is None:
        prompt = task_prompt
    else:
        prompt = task_prompt + text_input
    inputs = processor(text=prompt, images=image, return_tensors="pt")
    generated_ids = model.generate(
      input_ids=inputs["input_ids"].cuda(),
      pixel_values=inputs["pixel_values"].cuda(),
      max_new_tokens=1024,
      early_stopping=False,
      do_sample=False,
      num_beams=3,
    )
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
    parsed_answer = processor.post_process_generation(
        generated_text,
        task=task_prompt,
        image_size=(image.width, image.height)
    )

    return parsed_answer

In [6]:
import pandas as pd

In [14]:
df = pd.read_csv("/kaggle/input/train-set/test.csv")

In [15]:
df.head(50000)

,index,image_link,group_id,entity_name
0,0,https://m.media-amazon.com/images/I/110EibNycl...,156839,height
1,1,https://m.media-amazon.com/images/I/11TU2clswz...,792578,width
2,2,https://m.media-amazon.com/images/I/11TU2clswz...,792578,height
3,3,https://m.media-amazon.com/images/I/11TU2clswz...,792578,depth
4,4,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578,depth
...,...,...,...,...
49995,50042,https://m.media-amazon.com/images/I/51W2OtWGSf...,853009,height
49996,50043,https://m.media-amazon.com/images/I/51W2Qzmt5g...,311997,width
49997,50044,https://m.media-amazon.com/images/I/51W2UvfoTb...,521308,width
49998,50045,https://m.media-amazon.com/images/I/51W2W-DXsM...,181357,depth


In [16]:
url = "https://m.media-amazon.com/images/I/91nXSCxxxaL.jpg"
image = Image.open(requests.get(url, stream=True).raw)

In [17]:
task_prompt = '<OCR>'
run_example(task_prompt)

{'<OCR>': 'BettyDelightsCrockerSUPER MOIST"ARTIFICIALLYFLAVORED CHERRYCHERRY CHIPFLAVORED CHEESECAKE MIXThere\'s puddingin the mixPERIODECALIFORNIA1.0G1.6gBowl to oven in minutes!NET WT 15.5 OZ (454g)'}

In [18]:
df = df.iloc[68000:69001]
df

,index,image_link,group_id,entity_name
68000,68061,https://m.media-amazon.com/images/I/51pnmGhwAx...,801829,width
68001,68062,https://m.media-amazon.com/images/I/51pnwq5QdH...,764618,width
68002,68063,https://m.media-amazon.com/images/I/51pnySG2zP...,654649,item_weight
68003,68064,https://m.media-amazon.com/images/I/51po-ujHiq...,483370,width
68004,68065,https://m.media-amazon.com/images/I/51po-ujHiq...,483370,depth
...,...,...,...,...
68996,69057,https://m.media-amazon.com/images/I/51qoExFdCa...,860821,width
68997,69058,https://m.media-amazon.com/images/I/51qoExFdCa...,860821,depth
68998,69059,https://m.media-amazon.com/images/I/51qoJMQkjC...,145486,height
68999,69060,https://m.media-amazon.com/images/I/51qoQzTdjc...,997176,voltage


In [19]:
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
from tqdm import tqdm
import os

def process_image_url(url, task_prompt='<OCR>'):
    global image  # Declare image as global so run_example can access it
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()
        image = Image.open(BytesIO(response.content))

        # Ensure the image is in RGB mode
        if image.mode != "RGB":
            image = image.convert("RGB")

        result = run_example(task_prompt)  # No need to pass image here
        return result
    except Exception as e:
        return f"Error processing image: {str(e)}"

def process_chunk(chunk, chunk_id):
    tqdm.pandas(desc=f"Processing chunk {chunk_id}")
    chunk['ocr_output'] = chunk['image_link'].progress_apply(lambda url: process_image_url(url))

    # Save the processed chunk
    chunk.to_csv(f'processed_chunk_{chunk_id}.csv', index=False)
    print(f"Chunk {chunk_id} processed and saved.")

    return chunk

# Assuming your DataFrame is named 'df' and the column with image links is 'image_link'
chunk_size = 2000
num_chunks = (len(df) + chunk_size - 1) // chunk_size  # Calculate number of chunks

processed_chunks = []

for i in tqdm(range(num_chunks), desc="Processing chunks"):
    start_idx = i * chunk_size
    end_idx = min((i + 1) * chunk_size, len(df))

    chunk = df.iloc[start_idx:end_idx].copy()
    processed_chunk = process_chunk(chunk, i)
    processed_chunks.append(processed_chunk)

# Combine all processed chunks
final_df = pd.concat(processed_chunks, ignore_index=True)

# Save the final result
final_df.to_csv('final_processed_data.csv', index=False)
print("All chunks processed. Final result saved to 'final_processed_data.csv'")

# Display the first few rows of the final DataFrame
print(final_df[['image_link', 'ocr_output']].head())

# Clean up individual chunk files
for i in range(num_chunks):
    os.remove(f'processed_chunk_{i}.csv')
print("Individual chunk files cleaned up.")

Processing chunks: 100%|██████████| 1/1 [14:47<00:00, 887.74s/it]

Chunk 0 processed and saved.
All chunks processed. Final result saved to 'final_processed_data.csv'
                                          image_link  \
0  https://m.media-amazon.com/images/I/51pnmGhwAx...   
1  https://m.media-amazon.com/images/I/51pnwq5QdH...   
2  https://m.media-amazon.com/images/I/51pnySG2zP...   
3  https://m.media-amazon.com/images/I/51po-ujHiq...   
4  https://m.media-amazon.com/images/I/51po-ujHiq...   

                                          ocr_output  
0                             {'<OCR>': '24"59"37"'}  
1  {'<OCR>': 'N.U.R.S.E4.6 in/6.4 in/11.8 cm16.2 ...  
2  {'<OCR>': 'THELONEDiSNEYLEGORANGERAges/odados7...  
3                       {'<OCR>': '30 cm11.81 inch'}  
4                       {'<OCR>': '30 cm11.81 inch'}  
Individual chunk files cleaned up.


In [20]:
df

,index,image_link,group_id,entity_name
68000,68061,https://m.media-amazon.com/images/I/51pnmGhwAx...,801829,width
68001,68062,https://m.media-amazon.com/images/I/51pnwq5QdH...,764618,width
68002,68063,https://m.media-amazon.com/images/I/51pnySG2zP...,654649,item_weight
68003,68064,https://m.media-amazon.com/images/I/51po-ujHiq...,483370,width
68004,68065,https://m.media-amazon.com/images/I/51po-ujHiq...,483370,depth
...,...,...,...,...
68996,69057,https://m.media-amazon.com/images/I/51qoExFdCa...,860821,width
68997,69058,https://m.media-amazon.com/images/I/51qoExFdCa...,860821,depth
68998,69059,https://m.media-amazon.com/images/I/51qoJMQkjC...,145486,height
68999,69060,https://m.media-amazon.com/images/I/51qoQzTdjc...,997176,voltage
